# OTT Platforms Recommendation Systems 

### submitted by: Anurag Sen (as5864@srmist.edu.in) 

OTT platform recommendation system. It can suggest a new user a series/movie based on the movie weight rate by IMDB. It can perform personalized recommendation based on title similarity. Designed using selection criterion of IMBD and user specific recommendation system using Tfidfvectorizer and Cosine Similarity.

The program was built in Python:

1. IMDB weight rate criterion
2. Tfidvectorizer

In [1]:
#libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("C:\\Users\Anurag Sen\Desktop\Recomm Systems\movies_metadata (1).csv")
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### Data Preprocessing

In [3]:
data['genres'] = data['genres'].fillna('[]')  #filling empty entries

In [4]:
data.head(100)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,NaN,3000000,"[{'id': 18, 'name': 'Drama'}]",NaN,406,tt0113247,fr,La Haine,Aimlessly whiling away their days in the concr...,...,1995-05-31,0.0,98.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,Three Young Friends... One Last Chance.,La Haine,False,7.9,695.0
96,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45549,tt0111173,en,Shopping,"A dark, hip, urban story of a barren and anony...",...,1994-12-06,0.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No one leaves without paying...,Shopping,False,5.6,13.0
97,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",http://www.nickbroomfield.com/heidifleiss.html,63076,tt0113283,en,Heidi Fleiss: Hollywood Madam,A documentary crew from the BBC arrives in L.A...,...,1995-12-27,0.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Heidi Fleiss: Hollywood Madam,False,6.8,4.0
98,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,11062,tt0115907,en,City Hall,The accidental shooting of a boy in New York l...,...,1996-02-16,0.0,111.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It started with a shootout on a rainswept stre...,City Hall,False,6.0,67.0


In [5]:
data['genres'] = data['genres'].apply(literal_eval) #converting genres from string to a list

In [6]:
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
#Genres as a list and eliminating keys from the dictionary and keeping just the values

In [8]:
data['genres'] = data['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [9]:
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [10]:
#converting non-null vote_count and vote_average into integer

In [11]:
votecount = data[data['vote_count'].notnull()]

In [12]:
vote_count = votecount['vote_count'].astype('int')
vote_count

0        5415
1        2413
2          92
3          34
4         173
         ... 
45461       1
45462       3
45463       6
45464       0
45465       0
Name: vote_count, Length: 45460, dtype: int32

In [13]:
voteaverage = data[data['vote_average'].notnull()]

In [14]:
vote_average = voteaverage['vote_average'].astype('int')
vote_average

0        7
1        6
2        6
3        6
4        5
        ..
45461    4
45462    9
45463    3
45464    0
45465    0
Name: vote_average, Length: 45460, dtype: int32

### Top Movies Recommendation System

In [15]:
#sorting the average votes in Descending order and number of votes greater than 1000

In [16]:
top_movies = data.copy()

#### Sorting the vote_average in descending order 

In [17]:
top_movies1 = top_movies.sort_values('vote_average', ascending=False).head(250)

In [18]:
top_movies1

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
21642,False,NaN,0,[Documentary],NaN,320849,tt0886500,en,Ice Age Columbus: Who Were the First Americans?,Firmly rooted in the latest scientific discove...,...,2005-01-01,0.0,0.0,[],Released,NaN,Ice Age Columbus: Who Were the First Americans?,False,10.0,1.0
15710,False,NaN,0,[Documentary],NaN,96451,tt1587373,en,If God Is Willing and da Creek Don't Rise,"In 2006, director Spike Lee created an astonis...",...,2010-08-23,0.0,255.0,[],Released,NaN,If God Is Willing and da Creek Don't Rise,False,10.0,1.0
22396,False,NaN,0,[Documentary],NaN,72123,tt1341746,en,Meat the Truth,Meat the Truth is a high-profile documentary w...,...,2008-10-03,0.0,74.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Meat the Truth,False,10.0,1.0
22395,False,NaN,0,[Documentary],http://www.marvinhamlischmovie.com/,230864,tt3011874,en,Marvin Hamlisch: What He Did For Love,When Marvin Hamlisch passed away in August 201...,...,2013-10-12,0.0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Marvin Hamlisch: What He Did For Love,False,10.0,1.0
35343,False,NaN,300000,"[Comedy, Documentary, Music, TV Movie]",NaN,140595,tt0308213,en,Elaine Stritch: At Liberty,Judy at the Palace. Sinatra at Carnegie Hall. ...,...,2002-01-01,0.0,140.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Legendary performances come along so rarely.,Elaine Stritch: At Liberty,False,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35428,False,NaN,0,"[Drama, Romance]",http://www.kuraitokorode.com/,206155,tt0872007,ja,暗いところで待ち合わせ,"Michiru has lost her eyesight, and to make mat...",...,2006-09-05,0.0,130.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Waiting in the Dark,False,9.0,2.0
24882,False,NaN,0,[],NaN,273334,tt3575800,en,United States of Secrets (Part One): The Program,Last year Edward Snowden downloaded tens of th...,...,2014-05-13,0.0,114.0,[],Released,NaN,United States of Secrets (Part One): The Program,False,9.0,1.0
4183,False,NaN,0,"[Thriller, Drama]",NaN,88727,tt0157411,no,1732 Høtten,A few months after a girl with developmental d...,...,1998-12-26,0.0,100.0,"[{'iso_639_1': 'no', 'name': 'Norsk'}]",Released,NaN,Bloody Angels,False,9.0,2.0
36345,False,NaN,0,"[History, Drama]",NaN,359154,tt4699592,ab,13 dies de octubre,NaN,...,2015-09-10,0.0,0.0,"[{'iso_639_1': 'ca', 'name': 'Català'}, {'iso_...",Released,NaN,13 days of October,False,9.0,1.0


#### Selecting the vote_count greater than 1000 

In [19]:
top_movies2 = top_movies[top_movies['vote_count']>1000]

In [20]:
top_movies2

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,3.735540e+08,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,2.627972e+08,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
5,False,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,1.874368e+08,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,3.521940e+08,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0
15,False,NaN,52000000,"[Drama, Crime]",NaN,524,tt0112641,en,Casino,The life of the gambling paradise – Las Vegas ...,...,1995-11-22,1.161124e+08,178.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No one stays at the top forever.,Casino,False,7.8,1343.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43644,False,NaN,34000000,"[Action, Crime]",NaN,339403,tt3890160,en,Baby Driver,After being coerced into working for a crime b...,...,2017-06-28,2.245113e+08,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,All you need is one killer track.,Baby Driver,False,7.2,2083.0
44009,False,"{'id': 86066, 'name': 'Despicable Me Collectio...",80000000,"[Action, Animation, Adventure, Family, Comedy]",http://www.despicable.me,324852,tt3469046,en,Despicable Me 3,Gru and his wife Lucy must stop former '80s ch...,...,2017-06-15,1.020063e+09,96.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Oh brother.,Despicable Me 3,False,6.2,2002.0
44274,False,"{'id': 173710, 'name': 'Planet of the Apes (Re...",152000000,"[Drama, Science Fiction, War]",http://www.foxmovies.com/movies/war-for-the-pl...,281338,tt3450958,en,War for the Planet of the Apes,Caesar and his apes are forced into a deadly c...,...,2017-07-11,3.699080e+08,140.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,For freedom. For family. For the planet.,War for the Planet of the Apes,False,6.7,1675.0
44678,False,NaN,100000000,"[Action, Drama, History, Thriller, War]",http://www.dunkirkmovie.com/,374720,tt5013056,en,Dunkirk,The miraculous evacuation of Allied soldiers f...,...,2017-07-19,5.198769e+08,107.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The event that shaped our world,Dunkirk,False,7.5,2712.0


#### Vote_average in descending order || Vote_count greater than 1000 

In [21]:
top_movies2.sort_values('vote_average', ascending=False).head(250) 

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
314,False,NaN,25000000,"[Drama, Crime]",NaN,278,tt0111161,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,...,1994-09-23,28341469.0,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,False,8.5,8358.0
40251,False,NaN,0,"[Romance, Animation, Drama]",https://www.funimationfilms.com/movie/yourname/,372058,tt5311514,ja,君の名は。,High schoolers Mitsuha and Taki are complete s...,...,2016-08-26,355298270.0,106.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Your Name.,False,8.5,1030.0
834,False,"{'id': 230, 'name': 'The Godfather Collection'...",6000000,"[Drama, Crime]",http://www.thegodfather.com/,238,tt0068646,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",...,1972-03-14,245066411.0,175.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An offer you can't refuse.,The Godfather,False,8.5,6024.0
1152,False,NaN,3000000,[Drama],NaN,510,tt0073486,en,One Flew Over the Cuckoo's Nest,While serving time for insanity at a state men...,...,1975-11-18,108981275.0,133.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"If he's crazy, what does that make you?",One Flew Over the Cuckoo's Nest,False,8.3,3001.0
1176,False,"{'id': 119674, 'name': 'Psycho Collection', 'p...",806948,"[Drama, Horror, Thriller]",NaN,539,tt0054215,en,Psycho,When larcenous real estate clerk Marion Crane ...,...,1960-06-16,32000000.0,109.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The master of suspense moves his cameras into ...,Psycho,False,8.3,2405.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11353,False,NaN,76000000,"[Drama, Action, Thriller, Science Fiction]",http://www.universalstudiosentertainment.com/c...,9693,tt0206634,en,Children of Men,"In 2027, in a chaotic world in which humans ca...",...,2006-09-22,69959751.0,109.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The future's a thing of the past.,Children of Men,False,7.4,2120.0
24241,False,NaN,58800000,"[War, Action]",http://www.americansnipermovie.com,190859,tt2179136,en,American Sniper,U.S. Navy SEAL Chris Kyle takes his sole missi...,...,2014-12-11,542307423.0,133.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The most lethal sniper in U.S. history.,American Sniper,False,7.4,4600.0
13893,False,"{'id': 1241, 'name': 'Harry Potter Collection'...",250000000,"[Adventure, Fantasy, Family]",http://harrypotter.warnerbros.com/harrypottera...,767,tt0417741,en,Harry Potter and the Half-Blood Prince,"As Harry begins his sixth year at Hogwarts, he...",...,2009-07-07,933959197.0,153.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Dark Secrets Revealed,Harry Potter and the Half-Blood Prince,False,7.4,5435.0
23561,False,NaN,18000000,"[Drama, Comedy]",NaN,194662,tt2562232,en,Birdman,A fading actor best known for his portrayal of...,...,2014-08-27,103215094.0,119.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,or (The Unexpected Virtue of Ignorance),Birdman,False,7.4,4657.0


### IMDB Weighted Rating 

In [22]:
#Weighted Rating

#W = (Rv + Cm)/(v+m)

#W = weighted rating
#R = Ratings
#v = number of votes
#C = the mean vote accross the whole report
#m = minimum votes required to be in Top250

In [23]:
C = vote_average.mean()
C

5.244896612406511

In [24]:
m = vote_count.quantile(0.95)
m

434.0

In [25]:
top_movies['year'] = pd.to_datetime(top_movies['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [26]:
top_movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,NaT
45462,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,2011
45463,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,1917


In [27]:
top_movies3 = top_movies[(top_movies['vote_count'] >= m) & (top_movies['vote_count'].notnull()) & (top_movies['vote_average'].notnull())][['title','year','vote_count','vote_average','popularity','genres']]
top_movies3.shape

(2274, 6)

In [28]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    W = ((R*v) + (C*m))/(m+v)
    return W

In [29]:
top_movies3['IMDB weight_rate'] = top_movies3.apply(weighted_rating, axis=1)

In [30]:
top_movies3

,title,year,vote_count,vote_average,popularity,genres,IMDB weight_rate
0,Toy Story,1995,5415.0,7.7,21.9469,"[Animation, Comedy, Family]",7.517830
1,Jumanji,1995,2413.0,6.9,17.0155,"[Adventure, Fantasy, Family]",6.647694
5,Heat,1995,1886.0,7.7,17.9249,"[Action, Crime, Drama, Thriller]",7.240726
9,GoldenEye,1995,1194.0,6.6,14.686,"[Adventure, Action, Thriller]",6.238750
15,Casino,1995,1343.0,7.8,10.1374,"[Drama, Crime]",7.175962
...,...,...,...,...,...,...,...
44624,What Happened to Monday,2017,598.0,7.3,60.581223,"[Science Fiction, Thriller]",6.435741
44632,Atomic Blonde,2017,748.0,6.1,14.455104,"[Action, Thriller]",5.786028
44678,Dunkirk,2017,2712.0,7.5,30.938854,"[Action, Drama, History, Thriller, War]",7.188902
44842,Transformers: The Last Knight,2017,1440.0,6.2,39.186819,"[Action, Science Fiction, Thriller, Adventure]",5.978807


In [31]:
top_movies3 = top_movies3.sort_values('IMDB weight_rate', ascending=False)
top_movies3

,title,year,vote_count,vote_average,popularity,genres,IMDB weight_rate
314,The Shawshank Redemption,1994,8358.0,8.5,51.6454,"[Drama, Crime]",8.339318
834,The Godfather,1972,6024.0,8.5,41.1093,"[Drama, Crime]",8.281246
12481,The Dark Knight,2008,12269.0,8.3,123.167,"[Drama, Action, Crime, Thriller]",8.195622
2843,Fight Club,1999,9678.0,8.3,63.8696,[Drama],8.168877
292,Pulp Fiction,1994,8670.0,8.3,140.95,"[Thriller, Crime]",8.154359
...,...,...,...,...,...,...,...
18101,Jack and Jill,2011,619.0,4.0,6.62003,[Comedy],4.513091
21238,Sharknado,2013,484.0,3.8,4.92819,"[TV Movie, Horror]",4.483099
1491,Batman & Robin,1997,1447.0,4.2,17.0388,"[Action, Crime, Fantasy]",4.441087
28207,The Boy Next Door,2015,1034.0,4.1,10.2004,[Thriller],4.438478


In [32]:
#All the rating adjustments are general algorithmic ratings, BUT NOT USER SPECIFIC*

### Genre based division

In [33]:
genre_TM = top_movies3.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1,drop=True)
genre_TM.name = 'genre'
genre_top_movies = top_movies3.drop('genres',axis=1).join(genre_TM)
genre_top_movies = genre_top_movies.sort_values('IMDB weight_rate', ascending=False)
genre_top_movies.head(10)


,title,year,vote_count,vote_average,popularity,IMDB weight_rate,genre
314,The Shawshank Redemption,1994,8358.0,8.5,51.6454,8.339318,Crime
314,The Shawshank Redemption,1994,8358.0,8.5,51.6454,8.339318,Drama
834,The Godfather,1972,6024.0,8.5,41.1093,8.281246,Crime
834,The Godfather,1972,6024.0,8.5,41.1093,8.281246,Drama
12481,The Dark Knight,2008,12269.0,8.3,123.167,8.195622,Crime
12481,The Dark Knight,2008,12269.0,8.3,123.167,8.195622,Drama
12481,The Dark Knight,2008,12269.0,8.3,123.167,8.195622,Thriller
12481,The Dark Knight,2008,12269.0,8.3,123.167,8.195622,Action
2843,Fight Club,1999,9678.0,8.3,63.8696,8.168877,Drama
292,Pulp Fiction,1994,8670.0,8.3,140.95,8.154359,Crime


In [34]:
genre_top_movies['genre'].unique()

array(['Crime', 'Drama', 'Thriller', 'Action', 'Romance', 'Comedy',
       'History', 'War', 'Adventure', 'Mystery', 'Science Fiction',
       'Animation', 'Fantasy', 'Family', 'Horror', 'Western', 'Music',
       'Documentary', 'TV Movie'], dtype=object)

In [35]:
genre_top_movies[genre_top_movies.genre == "Animation"]

,title,year,vote_count,vote_average,popularity,IMDB weight_rate,genre
5481,Spirited Away,2001,3968.0,8.3,41.0489,7.998793,Animation
359,The Lion King,1994,5520.0,8.0,21.6058,7.799175,Animation
30315,Inside Out,2015,6737.0,7.9,23.9856,7.739309,Animation
9698,Howl's Moving Castle,2004,2049.0,8.2,16.136,7.683482,Animation
2884,Princess Mononoke,1997,2041.0,8.2,17.1667,7.681812,Animation
...,...,...,...,...,...,...,...
5346,Stuart Little 2,2002,632.0,5.4,9.27491,5.336853,Animation
18297,Alvin and the Chipmunks: Chipwrecked,2011,499.0,5.4,8.06639,5.327851,Animation
14562,Alvin and the Chipmunks: The Squeakquel,2009,688.0,5.3,9.90996,5.278685,Animation
7729,Garfield,2004,851.0,5.2,10.0037,5.215164,Animation


In [36]:
genre_top_movies[genre_top_movies.genre == "Family"]

,title,year,vote_count,vote_average,popularity,IMDB weight_rate,genre
5481,Spirited Away,2001,3968.0,8.3,41.0489,7.998793,Family
1225,Back to the Future,1985,6239.0,8.0,25.7785,7.820813,Family
359,The Lion King,1994,5520.0,8.0,21.6058,7.799175,Family
30315,Inside Out,2015,6737.0,7.9,23.9856,7.739309,Family
17437,Harry Potter and the Deathly Hallows: Part 2,2011,6141.0,7.9,24.9907,7.724743,Family
...,...,...,...,...,...,...,...
350,The Flintstones,1994,559.0,5.0,15.0196,5.107034,Family
29337,Paul Blart: Mall Cop 2,2015,576.0,5.0,17.0204,5.105233,Family
11445,Eragon,2006,990.0,4.9,9.85113,5.005116,Family
6417,Spy Kids 3-D: Game Over,2003,525.0,4.7,8.02515,4.946596,Family


In [37]:
genre_top_movies[genre_top_movies.genre == "Action"]

,title,year,vote_count,vote_average,popularity,IMDB weight_rate,genre
12481,The Dark Knight,2008,12269.0,8.3,123.167,8.195622,Action
15480,Inception,2010,14075.0,8.1,29.1081,8.014597,Action
1154,The Empire Strikes Back,1980,5998.0,8.2,19.471,8.000604,Action
7000,The Lord of the Rings: The Return of the King,2003,8226.0,8.1,29.3244,7.956915,Action
256,Star Wars,1977,6778.0,8.1,42.1497,7.928187,Action
...,...,...,...,...,...,...,...
1486,Speed 2: Cruise Control,1997,439.0,4.1,11.2685,4.669170,Action
7939,Catwoman,2004,833.0,4.2,13.3403,4.557920,Action
26563,Fantastic Four,2015,2322.0,4.4,17.2465,4.533050,Action
1491,Batman & Robin,1997,1447.0,4.2,17.0388,4.441087,Action


In [38]:
genre_top_movies[genre_top_movies.genre == "Drama"]

,title,year,vote_count,vote_average,popularity,IMDB weight_rate,genre
314,The Shawshank Redemption,1994,8358.0,8.5,51.6454,8.339318,Drama
834,The Godfather,1972,6024.0,8.5,41.1093,8.281246,Drama
12481,The Dark Knight,2008,12269.0,8.3,123.167,8.195622,Drama
2843,Fight Club,1999,9678.0,8.3,63.8696,8.168877,Drama
351,Forrest Gump,1994,8147.0,8.2,48.3072,8.050540,Drama
...,...,...,...,...,...,...,...
21848,The Counselor,2013,724.0,5.0,14.7236,5.091783,Drama
20598,Spring Breakers,2013,1580.0,5.0,14.8302,5.052773,Drama
11012,My Super Ex-Girlfriend,2006,453.0,4.8,12.8196,5.017683,Drama
32891,Knock Knock,2015,880.0,4.9,11.898793,5.013916,Drama


### Content Based Recommendation

In [39]:
#Customized Recommendation

In [40]:
links_small = pd.read_csv("C:\\Users\Anurag Sen\Desktop\Recomm Systems\links_small.csv")
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [41]:
top_movies = top_movies.drop([19730, 29503, 35587])

In [42]:
top_movies['id'] = top_movies['id'].astype('int')
top_movies4 = top_movies[top_movies['id'].isin(links_small)]
top_movies4.shape

(9099, 25)

In [43]:
top_movies4.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995


In [44]:
top_movies4['tagline'] = top_movies4['tagline'].fillna('')
top_movies4['description'] = top_movies4['overview'] + top_movies4['tagline']
top_movies4['description'] = top_movies4['description'].fillna('na')

In [45]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
tfidf_matrix = tf.fit_transform(top_movies4['description'])

In [46]:
tfidf_matrix

<9099x268124 sparse matrix of type '<class 'numpy.float64'>'
	with 540603 stored elements in Compressed Sparse Row format>

In [47]:
tfidf_matrix.shape

(9099, 268124)

#### Cosine similarity of words
##### It will return similar movies to the given title

In [48]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [49]:
cosine_sim

array([[1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
        0.        ],
       [0.00680476, 1.        , 0.01531062, ..., 0.00357057, 0.00762326,
        0.        ],
       [0.        , 0.01531062, 1.        , ..., 0.        , 0.00286535,
        0.00472155],
       ...,
       [0.        , 0.00357057, 0.        , ..., 1.        , 0.07811616,
        0.        ],
       [0.00344913, 0.00762326, 0.00286535, ..., 0.07811616, 1.        ,
        0.        ],
       [0.        , 0.        , 0.00472155, ..., 0.        , 0.        ,
        1.        ]])

In [50]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

Pairwise cosine similarity is noted. Now to make a function that returns 30 most similar movies based on cosine similarity

In [51]:
top_movies4 = top_movies4.reset_index()
titles = top_movies4['title']
indices = pd.Series(top_movies4.index, index = top_movies4['title'])

In [52]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key= lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [53]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

In [54]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [55]:
get_recommendations('Toy Story').head(10)

2502               Toy Story 2
7535               Toy Story 3
6193    The 40 Year Old Virgin
2547           Man on the Moon
6627              Factory Girl
4702    What's Up, Tiger Lily?
889      Rebel Without a Cause
6554    For Your Consideration
4988          Rivers and Tides
1599                 Condorman
Name: title, dtype: object